In [1]:
import warnings
warnings.filterwarnings(action='ignore') 

import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Okt
okt = Okt()

# 크롤링 데이터 인풋
data = pd.read_excel('210607_모더나_neg_data.xlsx', engine ='openpyxl')  
# del data['Unnamed: 1']
# data.columns=['vaccine_name','url','username','date','text','prepro_text','token','sentiment_score']
data.columns=['vaccine_name','url','username','date','text','prepro_text']

In [2]:
# 감성어 사전 인풋
n_dict = pd.read_excel('SentiWord_Dict.xlsx', engine ='openpyxl')
#n_dict = n_dict.drop(n_dict.columns[0],axis=1)

In [3]:
# tag에 따라 감성어 사전 구분 
from tqdm import tqdm
dict_emoji=[]
dict_zzgg=[]
dict_bbnn=[]
dict_length_one=[]
dict_length_two=[]
dict_length_thr=[]
dict_length_fou=[]
dict_length_fiv=[]
dict_hashtag=[]

for num in tqdm(range(len(n_dict['word']))):
    i = n_dict['word'][num]
    w = okt.pos(i)[0][1]
    if w=='Punctuation'or w=='Foreign'or w=='KoreanParticle':
        if i =='ㅋㄷ' or i == 'ㅎㅎ' :dict_zzgg.append(i)
        elif i=='ㅜ' or i=='ㅜ.ㅜ' or i=='ㅜ_ㅜ' or i=='ㅜㅜ' or i=='ㅜㅡ' or i=='ㅠ' or i=='ㅠ_ㅠ' or i=='ㅠㅠ' :dict_bbnn.append(i)
        else:dict_emoji.append(i)
    else :
        if len(i)==1 and i not in dict_length_one:dict_length_one.append(i)
        elif len(i)>=2:
            if i[0] != '#' and i[:2] not in dict_length_two :dict_length_two.append(i)
            elif i[0] != '#' and i[:3] not in dict_length_thr :dict_length_thr.append(i)
            elif i[0] != '#' and i[:4] not in dict_length_fou :dict_length_fou.append(i)
            elif i[0] != '#' and i[:5] not in dict_length_fiv :dict_length_fiv.append(i)
            elif i[0] == '#' and i[:2] not in dict_hashtag :dict_hashtag.append(i)

print(len(n_dict))
print(len(dict_emoji))
print(len(dict_zzgg))
print(len(dict_bbnn))
print(len(dict_length_one))
print(len(dict_length_two))
print(len(dict_length_thr))
print(len(dict_length_fou))
print(len(dict_length_fiv))
print(len(dict_hashtag))

100%|████████████████████████████████████████████████████████████████| 14855/14855 [00:11<00:00, 1283.84it/s]

14855
67
2
5
26
12073
2177
474
31
0


In [4]:
# 분석에 사용할 함수 선언
def wordTokenizer(sentence):
    tokenizedSentence=okt.morphs(sentence)
    return tokenizedSentence



#문장별 감성분석
def sc_calculator(sentence):
    sentimentScore= 0
    for token in sentence:
        for e in dict_emoji:
            if e in token:
                freq = len(token.split(e))-1
                score = int(n_dict.loc[n_dict['word']==e,'sentimentScore'])
                if freq>=2: 
                    if score*2>=4 or score*2<=-4:sentimentScore = sentimentScore + score
                    else:sentimentScore= sentimentScore + score*2
                else: sentimentScore = sentimentScore + score
        for zg in dict_zzgg:
            if zg in token:
                freq = len(token.split(zg))-1
                if freq>=2: sentimentScore = sentimentScore + 2
                else: sentimentScore +=1
                break
        for bn in dict_bbnn:
            if bn in token:
                freq = len(token.split(bn))-1
                if freq>=2: sentimentScore = sentimentScore - 2
                else: sentimentScore=sentimentScore-1
                break
        for o in dict_length_one:
            if o == token:
                score = int(n_dict.loc[n_dict['word']==o,'sentimentScore'])
                sentimentScore = sentimentScore + score
        for t in dict_length_two:
            if t in token and token[0]!='#': 
                freq = len(token.split(t))-1
                score = int(n_dict.loc[n_dict['word']==t,'sentimentScore'])
                sentimentScore = sentimentScore + freq*score
            #if token =='분위기':pass
        for t in dict_length_thr:
            if t in token and token[0]!='#': 
                freq = len(token.split(t))-1
                score = int(n_dict.loc[n_dict['word']==t,'sentimentScore'])
                sentimentScore = sentimentScore + freq*score
            #if token =='분위기':pass
        for t in dict_length_fou:
            if t in token and token[0]!='#': 
                freq = len(token.split(t))-1
                score = int(n_dict.loc[n_dict['word']==t,'sentimentScore'])
                sentimentScore = sentimentScore + freq*score
            #if token =='분위기':pass
        for t in dict_length_fiv:
            if t in token and token[0]!='#': 
                freq = len(token.split(t))-1
                score = int(n_dict.loc[n_dict['word']==t,'sentimentScore'])
                sentimentScore = sentimentScore + freq*score
            #if token =='분위기':pass    
        for h  in dict_hashtag:
            if h in token: 
                freq = len(token.split(h))-1
                score = int(n_dict.loc[n_dict['word']==h,'sentimentScore'])
                sentimentScore = sentimentScore + score
    return sentimentScore

In [5]:
# 불용어 제거 (숫자, 영어, 자음, 모음) 
# https://data-newbie.tistory.com/206
# https://github.com/lovit/soynlp/blob/master/tutorials/normalizer_usage.ipynb
# https://hashcode.co.kr/questions/692/%EC%8A%A4%ED%8A%B8%EB%A7%81%EC%97%90-%EB%AA%A8%EB%93%A0-%EA%B3%B5%EB%B0%B1-%EB%AC%B8%EC%9E%90%EB%A5%BC-%EC%A0%9C%EA%B1%B0%ED%95%98%EA%B3%A0-%EC%8B%B6%EC%9D%80%EB%8D%B0-%EC%95%9E-%EB%92%A4-%EA%B3%B5%EB%B0%B1%EB%A7%8C-%EC%A0%9C%EA%B1%B0%EB%90%A9%EB%8B%88%EB%8B%A4

import re
import sys
sys.path.insert(0, '../')
import soynlp
from soynlp.normalizer import *

final_only_sentence = []
for t in range(len(data)):
    # 특수문자제거 
    only_sentence = data['text'][t]
    only_ko_word1 = re.sub('[^ ㄱ-ㅣ가-힣]+','', str(only_sentence)) #한글빼고 다 제거 
    only_ko_word2 = emoticon_normalize(only_ko_word1, num_repeats=1) # ㅋㅋ,ㅜㅜ 자음,모음제거, 근데 ㅋㄱㅋ라던지 ㅠㅜㅠ 같은건 반복단어가 아니라 제거가 안돼서 한번 더 제거 필요
    pattern = '([ㄱ-ㅎㅏ-ㅣ])+'
    only_ko_word3 = re.sub(pattern=pattern, repl='', string=only_ko_word2)
    #print(only_ko_word3)
    final_only_sentence.append([data['vaccine_name'][t],data['url'][t],data['username'][t],data['date'][t],data['text'][t],only_ko_word3])
    #print(final_only_sentence)
    #final_only_sentence.append(,only_ko_word2)

In [6]:
# 제거된 특수문자를 dataframe으로 변경 
# 불용어 제거 print찍으면 메모리 많이 먹어서 아래 따로 실행
pd_final_only_sentence = pd.DataFrame(final_only_sentence)
pd_final_only_sentence.columns=['vaccine_name','url','username','date','text','prepro_text']
print(type(pd_final_only_sentence))

<class 'pandas.core.frame.DataFrame'>


In [7]:
pd_final_only_sentence

,vaccine_name,url,username,date,text,prepro_text
0,모더나,https://twitter.com/nowhere_6869/status/124119...,nowhere_6869,2020-03-21 02:39:57.000000,NIAID 소장은 트럼프 행정부의 바이러스 위기대응 프로그램에 핵심인물인 '앤서니 ...,소장은 트럼프 행정부의 바이러스 위기대응 프로그램에 핵심인물인 앤서니 파우치 박사다...
1,모더나,https://twitter.com/nowhere_6869/status/124119...,nowhere_6869,2020-03-21 02:41:14.000000,"또 하나 주목할 점은, 모더나(Moderna)의 ""면책 조항""이다. \n\n""사용이...",또 하나 주목할 점은 모더나의 면책 조항이다 사용이 승인된 기술을 이용한 상업제품은...
2,모더나,https://twitter.com/loremlumen/status/12464880...,loremlumen,2020-04-04 17:21:30.000000,"결국 백신이 나와야 종결될 것임. 현대 가장 진도 빠른 미 모더나, 내년 봄 내지 ...",결국 백신이 나와야 종결될 것임 현대 가장 진도 빠른 미 모더나 내년 봄 내지 여름...
3,모더나,https://twitter.com/agit00/status/124988698940...,agit00,2020-04-14 02:27:34.000000,"- ""이 업체는 코로나 백신 중 미국 모더나 세러퓨틱스에서 개발중인 ‘mRNA-12...",이 업체는 코로나 백신 중 미국 모더나 세러퓨틱스에서 개발중인 백신이 가장 빠르게 ...
4,모더나,https://twitter.com/jnjfilm/status/12499684964...,jnjfilm,2020-04-14 07:51:27.000000,사용한 AI 이름이 09냐? 인공지능이 예측한 코로나19 치료제·백신 출시일은? 치...,사용한 이름이 냐 인공지능이 예측한 코로나 치료제백신 출시일은 치료제는 년 백신은 ...
...,...,...,...,...,...,...
1202,모더나,https://twitter.com/minwoon/status/13705493300...,minwoon,2021-03-13 01:36:38.000000,"기저질환없고 건강했던 유타주 성형외과 의사 카시디 쿠릴(39), 모더나 백신 2차 ...",기저질환없고 건강했던 유타주 성형외과 의사 카시디 쿠릴 모더나 백신 차 접종 후 통...
1203,모더나,https://twitter.com/hankyungmedia/status/13705...,hankyungmedia,2021-03-13 03:59:59.999999,미국에서 코로나19 백신을 맞은 30대 여성이 숨졌습니다. 유타주 오그던에 살던 카...,미국에서 코로나 백신을 맞은 대 여성이 숨졌습니다 유타주 오그던에 살던 카시디 커릴...
1204,모더나,https://twitter.com/yesilovekallmin/status/137...,yesilovekallmin,2021-03-14 00:06:33.000000,오늘 모더나 1차 맞고옴. 서류준비 다해갔는데 신분증확인만 함.. 매우 허무함.. ...,오늘 모더나 차 맞고옴 서류준비 다해갔는데 신분증확인만 함 매우 허무함 차는 살짝 ...
1205,모더나,https://twitter.com/myfirstlove85/status/13711...,myfirstlove85,2021-03-14 18:37:06.999999,@angelesyogi 저도 1차접종때는 (모더나) 밤에 팔만 살짝 근육통 있었어요...,저도 차접종때는 모더나 밤에 팔만 살짝 근육통 있었어요 차접종때는 밤에 자다가 오한...


In [8]:
# okt를 활용한 형태소분석 실행

tokenized_data=[]

for i in range(len(pd_final_only_sentence)):
    if i%40000==0:print(i)
    if str(pd_final_only_sentence['text'][i])!='nan':
        tokenized_sentence = wordTokenizer(pd_final_only_sentence['prepro_text'][i]) # tokenized하는 부분
        #print(tokenized_sentence)
        tokenized_data.append([pd_final_only_sentence['vaccine_name'][i],pd_final_only_sentence['url'][i],pd_final_only_sentence['username'][i],pd_final_only_sentence['date'][i],pd_final_only_sentence['text'][i],pd_final_only_sentence['prepro_text'][i],tokenized_sentence])
print('finished')

0
finished


In [9]:
# okt 형태소분석 데이터에 대한 감성분석 실행

st_score_list=[]
for i in range(len(tokenized_data)):
    record=tokenized_data[i]
    sentence=record[-1]
    sentiment_score = sc_calculator(sentence)
    st_score_list.append([record[0],record[1],record[2],record[3],record[4],record[5],record[6],sentiment_score])
st_score_list_=pd.DataFrame(st_score_list)  
st_score_list_.columns=['vaccine_name','url','username','date','text','prepro_text','token','sentiment_score']

print('finished')

finished


In [10]:
total_token_data =st_score_list_['token']

In [11]:
stopwords=['건','에서','에','기있음','년','앤드류','태',"'",' ',',','파','슬로','스주','돼','덱','공','진','살','등','쿠팡','구','팡','했다고','미','변','후','주','차','아냐','못','건','명','했다','들','문','대요','까요','틸','않나요','아닙니다','플로','주교','존슨','준','월','중','백','번','유진','요크','나트','미론','로는','손','기','장','세','했','야','아니다','번만','좀','좀더','너무','은주',
           '봉','씨','좀','야','그렇게','켜','번','백','해주는','있는데','있어요','신','음','씨','세',
           '하는데','본','안되지','잔','기','유진','미론','면','토끼','요크','나트','라','회','층','아니고',
           '에요','대','전','죠','다음','네이버','출처','뉴스','은','는','의','에','로','으로','과','와','도',
           '만','에서','이나','나','까지','부터','에게',
           '께','처럼','이라도','라도','으로서','로서','조차','만큼','같이','마저','이나마','나마',
           '한테','더러','에게서','한테서','께서','이야','이라야','이',
            '같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문','그것','는',
            '\n','도','벌','수','있기','때문','는','도','에게','이기','때문','할','수록','로','을',
            '하는','...','........','하면','해주세요','•••','으로','어쩔','임',')','저','두','다',
            '이에요','!!','에는','있어서','(','가','정','한','요',',,)','이번','엔','만',
            '됨요','거여','후후','^^','?','제','가','냥냥','님','하','거든여',',,','5','..',
            '~','박','하기','.','뭐라','..?','을','가','하는','보다','것','이','를','게','그냥',
            '이러는','하','ㅠ','\n\n','는','글','고','됐다','뭘','뭐','든','은','아니면','풀','거기',
            '☆','아닝','상','그거','라고','그','어쩌구','아녀','??','과','샘','처럼','뭔가','적','아닌가',
            'n','년','만에','에','서','잘','같은데','확실','치','않다','....','아','됐지','바로','해야',
            '걸','쓰','읍','네','씩','나','부르','심',';;','네네','진짜','이라','이지','거','이었으면',
            '..............','.,,','했음','아니',',,,','ㅏ','.,','됨','청','뚝','일것','같음',
            '조','와','항','카','인','하나','안','날','1시간','밖에','에서','야할듯','인데','이고','안','내','과는',
            '이래','달','흑','!!!!!!!!','시','헙','아님','처','내','이지만','여기는','?...','뭐임',
            '저렇게','상관',',...','또','곧','ㅡ','"','.....','.......','하셔야죠','\\n','[',']',
            '같아요','해서','가져갑니다','!','근데','같아','의','라도'
          ]

In [12]:
# preprocessed_contents = 불용어 제거하고 들어간 real 콘텐츠
preprocessed_contents=[]

for sentence in total_token_data:
    terms =[] 
    for word in sentence:
        if word not in stopwords:terms.append(word) 
    preprocessed_contents.append(terms)

In [13]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)

for text in preprocessed_contents:
    for token in text:
        frequency[token] += 1

significant_contents=[]
for text in preprocessed_contents:
    terms=[]
    for token in text :
        if frequency[token] > 0:terms.append(token)
#         if frequency[token] > 2 and '.' not in token and '•' not in token:terms.append(token)
    significant_contents.append(terms)

In [14]:
significant_contents

[['소장',
  '트럼프',
  '행정부',
  '바이러스',
  '위기',
  '대응',
  '프로그램',
  '핵심',
  '인물',
  '앤서니',
  '파우치',
  '박사',
  '파우치',
  '게이츠',
  '모',
  '더',
  '코로나바이러스',
  '백신',
  '팀',
  '그리고',
  '연',
  '팀',
  '관해',
  '특히',
  '주목',
  '점',
  '불과',
  '몇',
  '전개',
  '되는',
  '이다'],
 ['주목',
  '점',
  '모',
  '더',
  '면책',
  '조항',
  '이다',
  '사용',
  '승인',
  '된',
  '기술',
  '이용',
  '상업',
  '제품',
  '존재',
  '하지',
  '않았다',
  '다시',
  '해',
  '사람',
  '건강',
  '안전',
  '대해',
  '전혀',
  '보증',
  '없다는',
  '점',
  '이다'],
 ['결국',
  '백신',
  '나와야',
  '종결',
  '될',
  '것임',
  '현대',
  '가장',
  '진도',
  '빠른',
  '모',
  '더',
  '내년',
  '봄',
  '내지',
  '여름',
  '출하',
  '예정',
  '파우치',
  '예측',
  '일치'],
 ['업체',
  '코로나',
  '백신',
  '미국',
  '모',
  '더',
  '세러퓨틱스',
  '개발',
  '백신',
  '가장',
  '빠르게',
  '개발',
  '되고',
  '있다고',
  '분석',
  '치료',
  '미국',
  '길',
  '리아드',
  '개발',
  '렘데',
  '시비르',
  '가장',
  '빠르다',
  '조선일보'],
 ['사용',
  '이름',
  '냐',
  '인공',
  '지능',
  '예측',
  '코로나',
  '치료',
  '백신',
  '출시일',
  '치료',
  '백신',
  '예상',
  '치료',
  '렘데',
  '시비르',
  '모',
  '더',
 

In [1]:
# 210624 분기별
# (2020-03-16 ~ 2020-06-15/ 2020-06-16 ~ 2020-09-15/ 2020-09-16 ~ 2020-12-15/ 2020-12-16 ~ 2021-03-15)
# 모더나 pos  [120,139,218,478]   총 955
# 모더나 neg  [108,91,134,875]
# 화이자 pos  [34,75,1015,1399]
# 화이자 neg [29,95,969,3097]
# 아스트라제네카 pos [7,65,343,1453]
# 아스트라제네카 neg [13,145,384,2221]
# 얀센 pos [56,50,61,91]
# 얀센 neg [44,81,66,74]
# vaccine_name	url	username	date	text	prepro_text	token	sentiment_score

In [15]:
# 아직 아래꺼 import 안됐는데 나중에 도움 될 수 있어서.. 참고자료 
# https://dacon.io/competitions/official/235590/codeshare/1078
import seaborn as sns, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import pandas as pd
from nltk.corpus import stopwords
from gensim.models.wrappers import DtmModel
from gensim.test.utils import common_corpus, common_dictionary
from pprint import pprint  # pretty-printer
from collections import defaultdict
from gensim import corpora

#DtmModel, common_corpusm common_dictionary 쓰려면 gensim 4.0.0b0 version

In [17]:
dictionary = corpora.Dictionary(significant_contents)
dictionary.save('210629_make_dictionary.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(6458 unique tokens: ['게이츠', '관해', '그리고', '대응', '더']...)


In [18]:
from gensim.test.utils import get_tmpfile
from gensim.corpora import MmCorpus

corpus = [dictionary.doc2bow(text) for text in significant_contents]
corpora.BleiCorpus.serialize('corpus.lda-c', corpus)
corpus = bleicorpus.BleiCorpus('corpus.lda-c')

In [19]:
import datetime
start = datetime.datetime.now()  # 시작 시간 저장
print(start)
# 210624 분기별
# (2020-03-16 ~ 2020-06-15/ 2020-06-16 ~ 2020-09-15/ 2020-09-16 ~ 2020-12-15/ 2020-12-16 ~ 2021-03-15)
# 모더나 pos  [120,139,218,478]   총 955
# 모더나 neg [108,91,134,875] 총 1,208
# 화이자 pos [34,75,1015,1399] 총 2,523
# 화이자 neg [29,95,969,3097] 총 4,190
# 아스트라제네카 pos [7,65,343,1453] 총 1,868
# 아스트라제네카 neg [13,145,384,2221] 총 2,763
# 얀센 pos [56,50,61,91]  총 258
# 얀센 neg [44,81,66,74] 총 265
time_slice = [108,91,134,875]

ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, 
                                 id2word=dictionary, time_slice=time_slice, num_topics=3)
finished = datetime.datetime.now()  # 시작 시간 저장
print(finished)
print(finished-start)

2021-07-01 13:40:10.927587
2021-07-01 13:51:32.987435
0:11:22.059848


In [20]:
topic = [0,1,2] # ldasequen 돌릴때 쓴 topic 개수를 넣기
topic_list = []
for t in topic:
    topic0 = ldaseq.print_topic_times(topic=t,top_terms=800)# topic = topic index, top_terms= 총 갯수
    topic_list.append(topic0) # topic_list에 topic & word & time_slice 넣기

In [21]:
topic_list

[[[('모', 0.05178632161780869),
   ('더', 0.05044794701746786),
   ('백신', 0.031115350246807786),
   ('코로나', 0.016705421701833742),
   ('항체', 0.012374622488638798),
   ('부작용', 0.006896354694612078),
   ('관련', 0.006226623729655568),
   ('치료', 0.005287685499031125),
   ('시비르', 0.005193900356479847),
   ('해', 0.0047858970829719235),
   ('주식', 0.004717960640616443),
   ('전원', 0.004691705113633223),
   ('임상시험', 0.004492937265786043),
   ('개발', 0.004475032141143017),
   ('렘데', 0.00443804854101818),
   ('형성', 0.0043169683370548025),
   ('미국', 0.0042750112621288095),
   ('바이오', 0.004193218765009594),
   ('사람', 0.004112383566833754),
   ('바이러스', 0.003986888425549355),
   ('효과', 0.0036240295757025555),
   ('결과', 0.003532282634082828),
   ('후보', 0.003342667320127235),
   ('화', 0.003308295222448649),
   ('오늘', 0.0028707159901875993),
   ('정도', 0.002812229973837692),
   ('증시', 0.0026980679365161075),
   ('재앙', 0.002592639662536849),
   ('하고', 0.002531906721810927),
   ('대한', 0.00252807300411205),
   (

In [207]:
# lda 모델 저장 
ldaseq.save('210701_neg_모더나.lda')
lda = ldaseq.load('210701_neg_모더나.lda')

In [208]:
# to excel 
q=1 

# textlist1=[] 
# numlist1=[]
topic_value_list = []
raw_data={}
raw_data_list=[]
for o in topic_list: #분기단위
    for pp in o: #토픽단위
        for u in pp:
            topic_value_list.append([u[0],u[1]])
#             textlist1.append(u[0])
#             numlist1.append(u[1])
        raw_data['quarter'+str(q)]=topic_value_list # quarter
        topic_value_list=[]
#        raw_data['value'+str(q)]=numlist1 # value
#         textlist1=[]
#         numlist1=[]
        q=q+1
    raw_data_list.append(raw_data)
    raw_data={}

raw_data_frame_list=[]

print(raw_data_list,"%%%")

for t in raw_data_list:
    
    raw_data_frame = pd.DataFrame(t) #데이터 프레임으로 전환 및 생성
    raw_data_frame_list.append(raw_data_frame)
    raw_data_frame=""
    

b=1

xlxs_dir='210627_neg_얀센_dtm.xlsx' #경로 및 파일명 설정
with pd.ExcelWriter(xlxs_dir) as writer:
    for y in raw_data_frame_list:
        
        y.to_excel(writer, sheet_name = 'topic'+str(b)) #raw_data1 시트에 저장
        b=b+1

[{'quarter1': [['얀', 0.04125772944225678], ['센', 0.021299100835917635], ['백신', 0.01734177137299757], ['센은', 0.007854102398024159], ['해', 0.006648701315666797], ['이랑', 0.0061703536860020294], ['쟝', 0.006045704368807668], ['센이', 0.00586222107184013], ['접종', 0.005781955405310878], ['계약', 0.005386927435094736], ['개', 0.004972574651169814], ['왜', 0.00494414166558776], ['자기', 0.004931815585689406], ['팔이', 0.00480570312996018], ['하고', 0.00479150898153681], ['한국', 0.004628432705196903], ['바이러스', 0.004215805024240044], ['같은', 0.0037141606104725963], ['보고', 0.0036971401849028836], ['이다', 0.003694751658731579], ['대통령', 0.0036324018996458395], ['기술', 0.003113760116795787], ['타이레놀', 0.0031052220480393155], ['한다', 0.003104007517683594], ['인간', 0.0030910163435577706], ['난', 0.003088257213135403], ['함', 0.0030875391201989023], ['내년', 0.003084744962754799], ['허가', 0.0030548732990332422], ['국민', 0.0030380504371191725], ['트', 0.0030035809813965053], ['미국', 0.002996991082394127], ['팜케', 0.0029705699984762

In [30]:
topic_ =pd.DataFrame(topic_list)
#topic__=topic_.transpose() # 행과 열 바꾸기
print(topic__)
# topic2_.to_csv('/users/kdh/desktop/data/non_face_class/result/neg/neg_topic2.csv')

NameError: name 'topic__' is not defined

In [64]:
topic__.to_excel('210625_pos_모더나_dtm_sample.xlsx')

In [23]:
# quarter1에서 topic 1,2,3,4,5,6 벨류 최고값만 남기고 중복제거 
a=0
overList1=[]
overList2=[]
testList=['a','b','c','d'] #예시 리스트 (4번돌리기 위한)
for j in testList:
    for i in topic_list:
        for k in i[a]:
            overList1.append(k[0])

    overList2.append(overList1)
    overList1=[]
    a=a+1

# print(overList2)


b=0
c=0
dic={}
overList4=[]
for q in testList:
    for w in topic_list:
        for e in w[b]:
            for h in overList2[b]:
                if(h==e[0]):
                    c=c+1
            if(c>1):
                
                if(dic.get(e[0])): #딕션어리에 키가 있을경우
                    if(dic.get(e[0]) < e[1]): #해당키의 벨류값과 이전에 포문으로 저장했던 딕션
                        dic[e[0]]=e[1] #덮어쓰기
                    else:
                        pass
                else: #딕션어리가 키가 없을경우 (처음에는 어차피 아무것도 없으므로 무조건 else로직을 탄다)
                    
                    dic[e[0]]=e[1] 
            c=0
                

    overList4.append(dic)
    dic={}
    b=b+1
    
print('@@@@@@@@@@@',overList4)

# z = 0
# for kyuli in overList4:
#     for kkk in kyuli:
#         z=z+1
#         #print("===========================")
# print(z)
finalList1=[] #제일 안쪽 배열
finalList2=[] #두번째 배열
finalList3=[] #제일 바깥 배열

index1=0
delNumList=[]
for x in topic_list:
    
    for y in x:

        for z in y:
            if(overList4[index1].get(z[0])): #포문이 돌때 해당키가 이전 최고 벨류를 뽑은 딕션어리에서 저장한 키값이 있을경우
                
                if(overList4[index1].get(z[0]) == z[1]):  #어차피 최고 벨류를 뽑은 딕션어리이므로 만약 포문이 돌때 해당키에 대한 벨류값이 같을경우 새로 만들 제일 안쪽 배열에 추가한다.
                    finalList1.append(z)
                else: #만약 최고 벨류를 뽑은 딕션어리 벨류값과 다르면 null처리를 한다.
                    finalList1.append(("NULL","NULL"))
                    delNumList.append("Null")
                    pass


            else: # 포문이 돌때 해당키가 이전 최고 벨류를 뽑은 딕션어리에서 저장한 키값이 없을경우
                finalList1.append(z)
        
        for del1 in delNumList:
            finalList1.remove(("NULL","NULL"))
        print(finalList1)
        for del2 in delNumList:
            finalList1.append(("",""))
        finalList2.append(finalList1) #새로 리스트 만드는작업 1
        finalList1=[] #초기화 작업
        index1=index1+1
        delNumList=[]
        if(index1==4):
            index1=0
    finalList3.append(finalList2) #새로 리스트 만드는작업 2
    finalList2=[] #초기화 작업
    

print("최종 완성본--> ",finalList3) #최종 완성된 리스트


#네번째 작업: 최종 완성된 리스트를 가지고 엑셀파일로 만든다. (엑셀쪽은 너가 고수라서 생략할께)
q=1 

# textlist1=[] 
# numlist1=[]
topic_value_list = []
raw_data={}
raw_data_list=[]
for o in finalList3: #월단위
    for pp in o: #세션단위
        for u in pp:
            topic_value_list.append([u[0],u[1]])
#             textlist1.append(u[0])
#             numlist1.append(u[1])
        raw_data['quarter'+str(q)]=topic_value_list # quarter
        topic_value_list=[]
#         raw_data['word'+str(q)]=textlist1
#         raw_data['value'+str(q)]=numlist1
#         textlist1=[]
#         numlist1=[]
        q=q+1
    raw_data_list.append(raw_data)
    raw_data={}

raw_data_frame_list=[]

print(raw_data_list,"%%%")

for t in raw_data_list:
    
    raw_data_frame = pd.DataFrame(t) #데이터 프레임으로 전환 및 생성
    raw_data_frame_list.append(raw_data_frame)
    raw_data_frame=""
    

b=1

xlxs_dir='210701_모더나_neg_중복제거후.xlsx' #경로 및 파일명 설정
with pd.ExcelWriter(xlxs_dir) as writer:
    for y in raw_data_frame_list:
        
        y.to_excel(writer, sheet_name = 'topic'+str(b)) #raw_data1 시트에 저장
        b=b+1

@@@@@@@@@@@ [{'모': 0.05527661944008681, '더': 0.056844725487429325, '백신': 0.058400205055030706, '코로나': 0.019547646564932783, '항체': 0.012374622488638798, '부작용': 0.015584974014781757, '관련': 0.006226623729655568, '치료': 0.005287685499031125, '해': 0.00620798974253938, '임상시험': 0.004492937265786043, '개발': 0.004475032141143017, '형성': 0.0043169683370548025, '미국': 0.00846755474231211, '바이오': 0.004193218765009594, '사람': 0.004112383566833754, '바이러스': 0.005076814272731065, '효과': 0.0046556589595331, '결과': 0.003532282634082828, '후보': 0.003342667320127235, '화': 0.003308295222448649, '오늘': 0.0028707159901875993, '정도': 0.002812229973837692, '재앙': 0.002592639662536849, '하고': 0.004134173227736303, '대한': 0.00252807300411205, '통화': 0.003818930238904203, '환자': 0.002500922612875466, '사': 0.0024764519811278736, '문재인': 0.004236988953115596, '화이': 0.0024044319962849563, '가장': 0.0023527033270528603, '성': 0.0022630310284814004, '자': 0.00579007294034676, '중국': 0.0021059726563286674, '기업': 0.0020971245713853957, '주가'

In [33]:
# visualization 
final_data = pd.read_excel('210629_모더나_neg_중복제거후.xlsx', engine ='openpyxl')  
del final_data['Unnamed: 0']
# data.columns=['vaccine_name','url','username','date','text','prepro_text','token','sentiment_score']
final_data.columns=['quarter1','quarter2','quarter3','quarter4']

In [89]:
final_data

,quarter1,quarter2,quarter3,quarter4
0,"['접종', 0.03224611197472766]","['접종', 0.032766154465717255]","['접종', 0.03370369055194211]","['접종', 0.03464192252373234]"
1,"['정부', 0.012751091323124993]","['정부', 0.0128141007760115]","['정부', 0.012974651516995648]","['정부', 0.013156337386596181]"
2,"['공급', 0.011328193953674579]","['공급', 0.01138399603234499]","['공급', 0.011563506760466155]","['공급', 0.011925886299627948]"
3,"['해', 0.008949078854575942]","['해', 0.009008196510608914]","['해', 0.009103501525514162]","['해', 0.009171911218670427]"
4,"['미국', 0.008254885468865143]","['미국', 0.00829389352563866]","['미국', 0.008338541991748608]","['명분', 0.008630270686453966]"
...,...,...,...,...
95,"['경험', 0.0012599188724160323]","['경험', 0.001258197278704709]","['경험', 0.0012574017276523325]","['경험', 0.0012549486669892812]"
96,"['도입', 0.0012537935463816702]","['도입', 0.0012506861941962027]","['도입', 0.0012441010883995922]","['도입', 0.0012364910808337424]"
97,"['선', 0.0012470510477537457]","['선', 0.001242491409455538]","['선', 0.0012342485341145697]","['알', 0.0012235318014809315]"
98,"['알', 0.0012335178667638755]","['알', 0.001228674392545005]","['알', 0.0012246577048934707]","['선', 0.001218744532366188]"


In [94]:
# visualization 
final_data = pd.read_excel('210630_모더나_neg_중복제거후.xlsx', engine ='openpyxl')  
del final_data['Unnamed: 0']
# data.columns=['vaccine_name','url','username','date','text','prepro_text','token','sentiment_score']
final_data.columns=['word1','value1','word2','value2','word3','value3','word4','value4']

In [120]:
final_data.iloc[:,4]

0      word3
1         명분
2         계약
3         확보
4       바이러스
       ...  
196      NaN
197      NaN
198      NaN
199      NaN
200      NaN
Name: word3, Length: 201, dtype: object

In [ ]:
# after labeling 